## Model Fit in Linear Regression - Lab

## Introduction
In this lab, you'll learn how to evaluate your model results, and you'll learn methods to select the appropriate features using stepwise selection.

## Objectives
You will be able to:
* Analyze the results of regression and R-squared and adjusted-R-squared 
* Understand and apply forward and backward predictor selection

## The Boston Housing Data once more

We pre-processed the Boston Housing Data the same way we did before:

- We dropped "ZN" and "NOX" completely
- We categorized "RAD" in 3 bins and "TAX" in 4 bins
- We used min-max-scaling on "B", "CRIM" and "DIS" (and logtransformed all of them first, except "B")
- We used standardization on "AGE", "INDUS", "LSTAT" and "PTRATIO" (and logtransformed all of them first, except for "AGE") 

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
boston = load_boston()

boston_features = pd.DataFrame(boston.data, columns = boston.feature_names)
boston_features = boston_features.drop(["NOX","ZN"],axis=1)

# first, create bins for based on the values observed. 3 values will result in 2 bins
bins = [0,6,  24]
bins_rad = pd.cut(boston_features['RAD'], bins)
bins_rad = bins_rad.cat.as_unordered()

# first, create bins for based on the values observed. 4 values will result in 3 bins
bins = [0, 270, 360, 712]
bins_tax = pd.cut(boston_features['TAX'], bins)
bins_tax = bins_tax.cat.as_unordered()

tax_dummy = pd.get_dummies(bins_tax, prefix="TAX")
rad_dummy = pd.get_dummies(bins_rad, prefix="RAD")
boston_features = boston_features.drop(["RAD","TAX"], axis=1)
boston_features = pd.concat([boston_features, rad_dummy, tax_dummy], axis=1)

age = boston_features["AGE"]
b = boston_features["B"]
logcrim = np.log(boston_features["CRIM"])
logdis = np.log(boston_features["DIS"])
logindus = np.log(boston_features["INDUS"])
loglstat = np.log(boston_features["LSTAT"])
logptratio = np.log(boston_features["PTRATIO"])

# minmax scaling
boston_features["B"] = (b-min(b))/(max(b)-min(b))
boston_features["CRIM"] = (logcrim-min(logcrim))/(max(logcrim)-min(logcrim))
boston_features["DIS"] = (logdis-min(logdis))/(max(logdis)-min(logdis))

#standardization
boston_features["AGE"] = (age-np.mean(age))/np.sqrt(np.var(age))
boston_features["INDUS"] = (logindus-np.mean(logindus))/np.sqrt(np.var(logindus))
boston_features["LSTAT"] = (loglstat-np.mean(loglstat))/np.sqrt(np.var(loglstat))
boston_features["PTRATIO"] = (logptratio-np.mean(logptratio))/(np.sqrt(np.var(logptratio)))

## Perform stepwise selection

The code for stepwise selection is copied below.

In [2]:
import statsmodels.api as sm

def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [3]:
result = stepwise_selection(boston_features, boston.target, verbose = True)
print('resulting features:')
print(result)

Add  LSTAT                          with p-value 9.27989e-122
Add  RM                             with p-value 1.98621e-16
Add  PTRATIO                        with p-value 2.5977e-12
Add  DIS                            with p-value 2.85496e-09
Add  B                              with p-value 2.77572e-06
Add  TAX_(0, 270]                   with p-value 0.000855799
Add  CHAS                           with p-value 0.00151282
Add  INDUS                          with p-value 0.00588575
resulting features:
['LSTAT', 'RM', 'PTRATIO', 'DIS', 'B', 'TAX_(0, 270]', 'CHAS', 'INDUS']


In [26]:
boston_features.columns


Index(['CRIM', 'INDUS', 'CHAS', 'RM', 'AGE', 'DIS', 'PTRATIO', 'B', 'LSTAT',
       'RAD_(0, 6]', 'RAD_(6, 24]', 'TAX_(0, 270]', 'TAX_(270, 360]',
       'TAX_(360, 712]'],
      dtype='object')

### Build the final model again in Statsmodels

In [52]:
ols1 = fin.join(boston_features)
ols1.head()

,MEDV,INDUS,CHAS,RM,DIS,PTRATIO,B,LSTAT,"TAX_(0, 270]"
0,24.0,-1.704344,0.0,6.575,0.542096,-1.443977,1.000000,-1.275260,0
1,21.6,-0.263239,0.0,6.421,0.623954,-0.230278,1.000000,-0.263711,1
2,34.7,-0.263239,0.0,7.185,0.623954,-0.230278,0.989737,-1.627858,1
3,33.4,-1.778965,0.0,6.998,0.707895,0.165279,0.994276,-2.153192,1
4,36.2,-1.778965,0.0,7.147,0.707895,0.165279,1.000000,-1.162114,1


In [59]:
from statsmodels.formula.api import ols
model = ols(formula= 'MEDV~INDUS+CHAS+RM+DIS+PTRATIO+B+LSTAT', data = ols1).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   MEDV   R-squared:                       0.773
Model:                            OLS   Adj. R-squared:                  0.770
Method:                 Least Squares   F-statistic:                     242.7
Date:                Sat, 20 Oct 2018   Prob (F-statistic):          4.89e-156
Time:                        22:00:39   Log-Likelihood:                -1464.7
No. Observations:                 506   AIC:                             2945.
Df Residuals:                     498   BIC:                             2979.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      5.0123      2.829      1.772      0.077      -0.545      10.570
INDUS         -1.2099      0.334     -3.619      0.000      -1.867      -0.553
CHAS           2.7988      0.795      3.519      0.000       1.236       4.362
RM             2.8712      0.388      7.405      0.000       2.109       3.633
DIS           -9.7229      1.326     -7.333      0.000     -12.328      -7.118
PTRATIO       -1.3564      0.227     -5.983      0.000      -1.802      -0.911
B              4.0619      0.934      4.347      0.000       2.226       5.898
LSTAT         -5.6444      0.320    -17.629      0.000      -6.274      -5.015
==============================================================================
Omnibus:                      105.185   Durbin-Watson:                   1.099
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              423.621
Skew:                           0.878   Prob(JB):                     1.03e-92
Kurtosis:                       7.124   Cond. No.                         96.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [56]:
from statsmodels.formula.api import ols
outcome = 'MEDV'
#boston_features = boston_features.drop(['CRIM', 'AGE','RAD_(0, 6]', 'RAD_(6, 24]', 'TAX_(270, 360]', 'TAX_(360, 712]'], axis = 1)
pred_sum = "+".join(boston_features.columns)
pred_sum
formula = outcome + "~" + pred_sum
#model = ols(formula= formula, data=boston_features).fit()
#model.summary()

Where our stepwise procedure mentions that "CHAS" was added with a p-value of 0.00151282, but our statsmodels output returns a p-value of 0.000. What is the intuition behind this?

## Use Feature ranking with recursive feature elimination

Use feature ranking to select the 5 most important features

In [61]:
#from sklearn.datasets import make_friedman1
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
selector = RFE(linreg, n_features_to_select = 5)
selector = selector.fit(boston_features, boston.target)
selector.support_
selector.ranking_

array([4, 1, 1, 1, 3, 1, 1, 2])

In [62]:
boston_features.columns

Index(['INDUS', 'CHAS', 'RM', 'DIS', 'PTRATIO', 'B', 'LSTAT', 'TAX_(0, 270]'], dtype='object')

Fit the linear regression model again using the 5 columns selected

In [75]:
#CHAS, RM, DIS, B, LSTAT
#boston_features = boston_features.drop(['INDUS', 'PTRATIO','TAX_(0, 270]'], axis = 1)
y = ols1.MEDV
linreg.fit(boston_features, y)
yhat = linreg.predict(boston_features)
type(yhat)

numpy.ndarray

Now, predict $\hat y$ using your model. you can use `.predict()` in scikit-learn

In [79]:
import numpy as np

def sq_err(ys_a, ys_b):
    errs = []
    for i in range(len(ys_a)):
        errs.append((ys_a[i]-ys_b[i])**2)
    sq_err = sum(errs)
    return sq_err

def r_squared(ys_real, ys_predicted):
    y_bar = np.mean(ys_real)
    y_exp = []
    for i in range(len(ys_real)):
        y_exp.append(y_bar)
    SSE = sq_err(ys_real, ys_predicted)
    SST = sq_err(ys_real, y_exp)
    R = 1 - (SSE/SST)
    return R
R = r_squared(y, yhat)

Now, using the formulas of R-squared and adjusted-R-squared below, and your Python/numpy knowledge, compute them and contrast them with the R-squared and adjusted-R-squared in your statsmodels output using stepwise selection. Which of the two models would you prefer?

$SS_{residual} = \sum (y - \hat{y})^2 $

$SS_{total} = \sum (y - \bar{y})^2 $

$R^2 = 1- \dfrac{SS_{residual}}{SS_{total}}$

$R^2_{adj}= 1-(1-R^2)\dfrac{n-1}{n-p-1}$

In [ ]:
# R^2 = 0.7429807743129865

In [81]:
N = len(y)
Ra = 1 - (1-R)*((N-1)/(N-5-1))
Ra

0.7404105820561163

In [ ]:
# values from the stepwise model were higher, so I'd use that model.

## Level up - Optional

- Perform variable selection using forward selection, using this resource: https://planspace.org/20150423-forward_selection_with_statsmodels/. Note that this time features are added based on the adjusted-R-squared!
- Tweak the code in the `stepwise_selection()`-function written above to just perform forward selection based on the p-value.

## Summary
Great! You now performed your own feature selection methods!